In [23]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('localhost', 27017)  # Assuming MongoDB is running locally on default port
db = client['bdaProject']  # Replace 'your_database_name' with your actual database name
collection = db['extractedFeatures']  # Replace 'your_collection_name' with your actual collection name

# Fetch specific fields from all documents in the collection
cursor = collection.find({}, {'track_id': 1, 'duration': 1, 'mfcc': 1, 'mfcc_normalized': 1, 'PCA1': 1, 'PCA2': 1})

# Convert documents to a list of dictionaries
data = list(cursor)

# Close the cursor
cursor.close()

# Convert list of dictionaries to pandas DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
print(df)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 13 but got size 1 for tensor number 2 in the list.

In [17]:
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
class MusicDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = {
            'track_id': torch.tensor(self.df.iloc[idx]['track_id'], dtype=torch.long),
            'mfcc': torch.tensor(self.df.iloc[idx]['mfcc'], dtype=torch.float),
            'mfcc_normalized': torch.tensor(self.df.iloc[idx]['mfcc_normalized'], dtype=torch.float),
            'PCA1': torch.tensor(self.df.iloc[idx]['PCA1'], dtype=torch.float),
            'PCA2': torch.tensor(self.df.iloc[idx]['PCA2'], dtype=torch.float),
        }
        return sample

# Create PyTorch Dataset
dataset = MusicDataset(df)



In [82]:
# Define model architecture
class RecommenderModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(RecommenderModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 128)  # Adjust the input dimension here
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Adjust the input dimension based on the size of the concatenated tensor
input_dim = 28  # Adjust this based on the actual size of the concatenated tensor x
model = RecommenderModel(input_dim)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [83]:
print(mfcc.shape)
print(mfcc_normalized.shape)
print(PCA1.shape)
print(PCA2.shape)


torch.Size([1, 32, 13])
torch.Size([1, 32, 13])
torch.Size([1, 32, 13])
torch.Size([1, 32, 13])


In [71]:
PCA1 = PCA1.expand(-1, -1, 13)
PCA2 = PCA2.expand(-1, -1, 13)

print(mfcc.shape)
print(mfcc_normalized.shape)
print(PCA1.shape)
print(PCA2.shape)

torch.Size([1, 32, 13])
torch.Size([1, 32, 13])
torch.Size([1, 32, 13])
torch.Size([1, 32, 13])


In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()

        # Concatenate features along the feature dimension
        x = torch.cat([batch['mfcc'], batch['mfcc_normalized'], batch['PCA1'].unsqueeze(1), batch['PCA2'].unsqueeze(1)], dim=1)

        # Pass concatenated features through the model
        outputs = model(x)

        # Reshape outputs to have a shape of [batch_size, 1]
        outputs = outputs.view(-1, 1)

        # Compute similarity scores between tracks
        sim_scores = cosine_similarity(outputs.detach().numpy())

        # Define target variable (similarity score)
        target = torch.tensor(sim_scores, dtype=torch.float)

        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}')


In [92]:
# Assuming 'model' is your trained model and 'dataloader' is your DataLoader for the evaluation set

model.eval()  # Set the model to evaluation mode

# Initialize a list to store the losses for each batch
batch_losses = []

with torch.no_grad():  # No need to track gradients during evaluation
    for batch in dataloader:
        # Concatenate features along the feature dimension
        x = torch.cat([batch['mfcc'], batch['mfcc_normalized'], batch['PCA1'].unsqueeze(1), batch['PCA2'].unsqueeze(1)], dim=1)
        
        # Pass concatenated features through the model
        outputs = model(x)
        
        # Reshape outputs to have a shape of [batch_size, 1]
        outputs = outputs.view(-1, 1)
        
        # Compute similarity scores between tracks
        sim_scores = cosine_similarity(outputs.detach().numpy())
        
        # Extract the diagonal of the similarity matrix to get the self-similarity scores
        true_sim = np.diag(sim_scores)
        
        # Flatten the model outputs to a vector
        pred_sim = outputs.squeeze().detach().numpy()
        
        # Compute Mean Squared Error for the current batch
        mse_loss = np.mean((true_sim - pred_sim) ** 2)
        
        # Store the loss for the current batch
        batch_losses.append(mse_loss)

# Compute the average loss across all batches
average_loss = np.mean(batch_losses)
print(f'Average Mean Squared Error: {average_loss}')


Average Mean Squared Error: 0.024993423372507095
